In [1]:
import ast
import pandas as pd

df = pd.read_csv('../data/computer_scientists.pp.csv')

df['name'] = df['name'].apply(lambda x: x.split(' ')[-1])
df['education'] = df['education'].apply(ast.literal_eval).apply(lambda x: x[0] if x else None)
df['awards'] = df['awards'].apply(ast.literal_eval).apply(lambda x: len(x))

df = df.dropna(subset=["education"], ignore_index=True)

df

,name,education,awards
0,Khan,University of Malaya,0
1,Aaronson,Cornell University,4
2,Abebe,Cornell University,0
3,Abelson,Princeton University,1
4,Abiteboul,University of Southern California,6
...,...,...,...
523,Zedan,University of Bristol,0
524,Zilberstein,"University of California, Berkeley",0
525,Zimmerman,Purdue University,0
526,Zuckerberg,Harvard University,0


In [2]:
import ipywidgets as widgets
from chord_dht.chord import Chord

m_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=5,
    step=1,
    description='M bits:',
    disabled=False,
    continuous_update=True
)

chord: Chord | None


@widgets.interact(m=m_slider)
def init_chord_ring(m: int):
    global chord
    chord = Chord(m)

    for i in range(2 ** m):
        chord.join(i)

    for _, row in df.iterrows():
        chord.insert(row['education'], {'name': row['name'], 'awards': row['awards']})

    return f'Chord ring initialized with {2 ** m} nodes'

interactive(children=(IntSlider(value=2, description='M bits:', max=5, min=1), Output()), _dom_classes=('widge…

In [6]:
education_dropdown = widgets.Dropdown(
    options=df['education'].unique().tolist(),
    description='Education:',
    continuous_update=True,
    disabled=False
)

awards_range_slider = widgets.IntRangeSlider(
    value=[0, df['awards'].max()],
    min=0,
    max=(df['awards'].max()),
    step=1,
    description='Awards:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)


@widgets.interact(education=education_dropdown, awards=awards_range_slider)
def lookup_scientists(education: str, awards: tuple[int, int]):
    scientists = chord.lookup(education)

    scientists = [scientist for scientist in scientists if awards[0] <= int(scientist['awards']) <= awards[1]]

    return pd.DataFrame(scientists) if scientists else 'No scientists found'

interactive(children=(Dropdown(description='Education:', options=('University of Malaya', 'Cornell University'…